In [102]:
import pandas as pd

In [103]:
df = pd.read_csv('CSV/diabetes.csv')

In [104]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [105]:
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import StandardScaler

In [106]:
train_x, test_x, train_y, test_y = split(df.drop(columns=['Outcome']), df['Outcome'], test_size=0.2)

In [107]:
scaler = StandardScaler()

In [108]:
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [109]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import kerastuner as kt

In [110]:
def build_model(hp):

    counter=0

    model = Sequential()
    
    

    for i in range(hp.Int('num_layers', 2, 20)):
        if counter == 0:
        
            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128),
                    activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                    )
                )
        else:
            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128),
                    activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']))
                )
            counter+=1
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer=hp.Choice('optimiser', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']), 
        loss='binary_crossentropy', metrics=['accuracy'])

        return model

In [111]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [112]:
tuner.search(train_x, train_y, epochs=5, validation_data=(test_x, test_y))

In [113]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9, 'units0': 24, 'activation0': 'sigmoid', 'optimiser': 'adam'}

In [114]:
model = tuner.get_best_models(num_models=1)[0]

/home/user/machine-learning/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/user/machine-learning/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [115]:
model.fit(train_x, train_y, initial_epoch=5, epochs=100)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7275 - loss: 0.5991
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7313 - loss: 0.5731
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7106 - loss: 0.5771  
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7446 - loss: 0.5450 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7124 - loss: 0.5572
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7318 - loss: 0.5321
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7266 - loss: 0.5397
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7259 - loss: 0.5352
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7138 - loss: 0.5512
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7266 - loss: 0.5322  
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7551 - loss: 0.5130 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

In [116]:
from sklearn.metrics import accuracy_score as score
# Suggested code may be subject to a license. Learn more: ~LicenseLog:3126165467.
# Suggested code may be subject to a license. Learn more: ~LicenseLog:3332693275.
import numpy as np

In [117]:
prob= model.predict(test_x)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [118]:
predict=np.where(prob>0.5, 1, 0)

In [119]:
score(test_y, predict)

0.7922077922077922